# 2024년 vs 2021 종별코드명 활용하여 상급, 종합 병원 등의 분포 확인

In [2]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [3]:
DATA_PATH = '/Users/haeunpark/mid_project_data'

In [4]:
# 2024 데이터 출력
pd.set_option('display.max_columns', None)


df_2024 = pd.read_csv(f'{DATA_PATH}/2024의원통합데이터.csv', index_col=0)
df_2021 = pd.read_csv(f'{DATA_PATH}/2021의원통합데이터.csv', index_col=0)

In [5]:
# 2024년도 총 병원 수 계산
df_2024_unique = df_2024.drop_duplicates(subset=['암호화요양기호'])
df_2021_unique = df_2021.drop_duplicates(subset=['암호화요양기호'])

total_hospitals_2024 = len(df_2024_unique)

# 2021년도 총 병원 수 계산
total_hospitals_2021 = len(df_2021_unique)

# 2024년도와 2021년도 총 병원 수의 증감량 계산
total_hospitals_diff = total_hospitals_2024 - total_hospitals_2021

# 결과 출력
print(f"2024년도 총 병원 수: {total_hospitals_2024}")
print(f"2021년도 총 병원 수: {total_hospitals_2021}")
print(f"2021년 대비 2024년 총 병원 수 증감량: {total_hospitals_diff}")

2024년도 총 병원 수: 78126
2021년도 총 병원 수: 74756
2021년 대비 2024년 총 병원 수 증감량: 3370


In [37]:
# 중복 제거: 암호화요양기호(병원 고유 식별자)를 기준으로 중복된 병원 제거
df_2024_unique = df_2024.drop_duplicates(subset=['암호화요양기호'])
df_2021_unique = df_2021.drop_duplicates(subset=['암호화요양기호'])

# 시도별 병원 유형 분포 집계 (2024년도 데이터)
hospital_2024_by_region = df_2024_unique.groupby(['시도코드명', '종별코드명']).size().unstack(fill_value=0)

# 시도별 병원 유형 분포 집계 (2021년도 데이터)
hospital_2021_by_region = df_2021_unique.groupby(['시도코드명', '종별코드명']).size().unstack(fill_value=0)

# # 서울, 경기, 인천을 가장 앞으로 두고 나머지 시도는 뒤에 위치하도록 설정
desired_order_2024 = ['서울', '경기', '인천'] + [x for x in hospital_2024_by_region.index if x not in ['서울', '경기', '인천']]
hospital_2024_by_region = hospital_2024_by_region.reindex(desired_order_2024)

desired_order_2021 = ['서울', '경기', '인천'] + [x for x in hospital_2021_by_region.index if x not in ['서울', '경기', '인천']]
hospital_2021_by_region = hospital_2021_by_region.reindex(desired_order_2021)

# 2021년 대비 2024년 병원 유형별 증감량 계산
hospital_diff_by_region = hospital_2024_by_region - hospital_2021_by_region


# 멀티 인덱스를 해제하여 종별코드명을 열로 이동시키고, 시도코드명을 인덱스로 설정
hospital_diff_by_region = hospital_diff_by_region.reset_index()

# 순차적인 숫자 인덱스를 사용하려면 인덱스를 재설정
hospital_diff_by_region.reset_index(drop=True, inplace=True)


# 결과 출력
hospital_diff_by_region



종별코드명,시도코드명,병원,보건소,보건의료원,보건지소,보건진료소,상급종합,요양병원,의원,정신병원,조산원,종합병원,치과병원,치과의원,한방병원,한의원
0,서울,4,0,0,2,0,0,-15,822,0,0,2,-4,-4,16,27
1,경기,11,1,0,-4,-8,1,-27,810,2,-1,4,6,268,37,110
2,인천,3,0,0,0,-1,0,-4,188,1,0,1,4,47,5,23
3,강원,-1,0,0,-1,0,0,-2,31,0,0,1,0,21,1,2
4,경남,-6,1,0,-1,-1,0,-14,45,3,0,0,2,26,8,-18
5,경북,-3,-1,0,-7,-10,0,-9,27,2,0,1,-2,20,2,-6
6,광주,0,0,0,1,0,0,1,96,2,0,1,-1,9,0,6
7,대구,1,1,0,6,10,0,-1,143,0,0,1,-2,42,9,20
8,대전,2,0,0,0,-1,1,-2,24,0,0,-1,1,11,1,-6
9,부산,9,0,0,0,0,1,-8,172,-1,0,0,1,32,9,9


In [41]:
hospital_diff_by_region.to_csv('/Users/haeunpark/mid_project_data/2021년대비2024년_지역별_병원유형_증감량.csv', index=False)

In [65]:
# # 병원 유형 분류 사전
# hospital_groups = {
#     '상급종합': '3차 의료',
#     '종합병원': '2차 의료',
#     '병원': '2차 의료',
#     '치과병원': '2차 의료',
#     '한방병원': '2차 의료',
#     '요양병원': '2차 의료',
#     '정신병원': '2차 의료',
#     '보건의료원': '2차 의료',
#     '의원': '1차 의료',
#     '치과의원': '1차 의료',
#     '한의원': '1차 의료',
#     '보건소': '1차 의료',
#     '보건지소': '1차 의료',
#     '보건진료소': '1차 의료',
#     '조산원': '특수의료'
# }


# # 병원 유형 분류 사전
hospital_groups = {
    '상급종합': '3차 의료',
    '종합병원': '2차 의료',
    '병원': '2차 의료',
    '치과병원': '2차 의료',
    '한방병원': '2차 의료',
    '요양병원': '2차 의료',
    '정신병원': '2차 의료',
    '보건의료원': '2차 의료',
    '의원': '1차 의료',
    '치과의원': '1차 의료',
    '한의원': '1차 의료',
    '조산원': '특수의료'
}


# 중복 제거: 암호화요양기호를 기준으로 중복된 병원 제거
df_2024_unique = df_2024.drop_duplicates(subset=['암호화요양기호'])
df_2021_unique = df_2021.drop_duplicates(subset=['암호화요양기호'])

# 병원 유형에 따른 1차, 2차, 3차 의료 수준 분류 추가
df_2024_unique['의료수준'] = df_2024_unique['종별코드명'].map(hospital_groups)
df_2021_unique['의료수준'] = df_2021_unique['종별코드명'].map(hospital_groups)


# 시도별 병원 유형 분포 집계 (2024년도 데이터)
hospital_2024_by_region_level = df_2024_unique.groupby(['시도코드명', '의료수준']).size().unstack(fill_value=0)


# 시도별 병원 유형 분포 집계 (2021년도 데이터)
hospital_2021_by_region_level = df_2021_unique.groupby(['시도코드명', '의료수준']).size().unstack(fill_value=0)

# # 시도별 병원 유형 증감량 계산 (2024년도 - 2021년도)
hospital_diff_by_region_level = hospital_2024_by_region_level - hospital_2021_by_region_level

# 결측값(NaN)은 0으로 대체 (변화가 없거나 새로운 유형일 경우)
hospital_diff_by_region_level = hospital_diff_by_region_level.fillna(0)

# 멀티 인덱스를 해제하여 종별코드명을 열로 이동시키고, 시도코드명을 인덱스로 설정
hospital_diff_by_region_level = hospital_diff_by_region_level.reset_index()

# 순차적인 숫자 인덱스를 사용하려면 인덱스를 재설정
hospital_diff_by_region_level.reset_index(drop=True, inplace=True)

# 결과 출력
hospital_diff_by_region_level

/var/folders/zj/p3cl2py51ldfrgsbtfp578x40000gn/T/ipykernel_41520/1582399177.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zj/p3cl2py51ldfrgsbtfp578x40000gn/T/ipykernel_41520/1582399177.py:44: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



의료수준,시도코드명,1차 의료,2차 의료,3차 의료,특수의료
0,강원,54,-1,0,0
1,경기,1188,33,1,-1
2,경남,53,-7,0,0
3,경북,41,-9,0,0
4,광주,111,3,0,0
5,대구,205,8,0,0
6,대전,29,1,1,0
7,부산,213,10,1,0
8,서울,845,3,0,0
9,세종시,55,1,0,0


In [57]:
hospital_diff_by_region_level.to_csv('/Users/haeunpark/mid_project_data/2021년대비2024년_지역별_1차_2차_3차병원수준_증감량_1차보건관련제외.csv', index=False)

In [66]:

# 병원 유형별 변화량 그래프 생성
fig = px.bar(hospital_diff_by_region_level.reset_index(),
             x='시도코드명',  # 시도코드명
             y=['1차 의료', '2차 의료', '3차 의료'],  # 의료 수준에 따른 변화량
             title='2021년 대비 2024년 시도별 1차, 2차, 3차 병원 유형별 변화량',
             barmode='group')  # 그룹화된 막대 그래프

# 그래프 표시
fig.show()

In [67]:
# 시도코드명을 '수도권'과 '비수도권'으로 분류하는 함수
def categorize_region(region):
    if region in ['서울', '경기', '인천']:
        return '수도권'
    else:
        return '비수도권'

# 2024년 데이터에 '지역' 분류 추가
df_2024_unique['지역'] = df_2024_unique['시도코드명'].apply(categorize_region)

# 2021년 데이터에 '지역' 분류 추가
df_2021_unique['지역'] = df_2021_unique['시도코드명'].apply(categorize_region)

# 병원 유형에 따른 1차, 2차, 3차 의료 수준 분류 추가
df_2024_unique['의료수준'] = df_2024_unique['종별코드명'].map(hospital_groups)
df_2021_unique['의료수준'] = df_2021_unique['종별코드명'].map(hospital_groups)

# 수도권 및 비수도권별 병원 유형 분포 집계 (2024년도 데이터)
hospital_2024_by_region_level = df_2024_unique.groupby(['지역', '의료수준']).size().unstack(fill_value=0)



# 수도권 및 비수도권별 병원 유형 분포 집계 (2021년도 데이터)
hospital_2021_by_region_level = df_2021_unique.groupby(['지역', '의료수준']).size().unstack(fill_value=0)

# 수도권 및 비수도권별 병원 유형 증감량 계산 (2024년도 - 2021년도)
hospital_diff_by_region_level = hospital_2024_by_region_level - hospital_2021_by_region_level

# 결측값(NaN)은 0으로 대체 (변화가 없거나 새로운 유형일 경우)
hospital_diff_by_region_level = hospital_diff_by_region_level.fillna(0)

# 결과 출력
print("2021년 대비 2024년 수도권 및 비수도권 병원 유형별 증감량:")
hospital_diff_by_region_level = hospital_diff_by_region_level.reset_index()
hospital_diff_by_region_level

2021년 대비 2024년 수도권 및 비수도권 병원 유형별 증감량:


/var/folders/zj/p3cl2py51ldfrgsbtfp578x40000gn/T/ipykernel_41520/4086575395.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zj/p3cl2py51ldfrgsbtfp578x40000gn/T/ipykernel_41520/4086575395.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zj/p3cl2py51ldfrgsbtfp578x40000gn/T/ipykernel_41520/4086575395.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

의료수준,지역,1차 의료,2차 의료,3차 의료,특수의료
0,비수도권,1046,1,1,0
1,수도권,2291,46,1,-1


In [68]:
hospital_diff_by_region_level.to_csv('/Users/haeunpark/mid_project_data/2021년대비2024년_수도권_비수도권_1차_2차_3차병원수준_증감량_1차보건제외.csv', index=False)

In [70]:
# 시도별 총의사수 집계 (2024년도 데이터)
total_doctors_2024 = df_2024_unique.groupby('시도코드명')['총의사수'].sum()

# 시도별 총의사수 집계 (2021년도 데이터)
total_doctors_2021 = df_2021_unique.groupby('시도코드명')['총의사수'].sum()

# 시도별 총의사수 변화량 계산 (2024년도 - 2021년도)
total_doctors_diff = total_doctors_2024 - total_doctors_2021

# 결측값(NaN)을 0으로 대체 (의사 수가 없는 경우)
total_doctors_diff = total_doctors_diff.fillna(0)

total_doctors_diff = total_doctors_diff.reset_index()

# 결과 출력
total_doctors_diff

,시도코드명,총의사수
0,강원,-242
1,경기,2033
2,경남,-130
3,경북,19
4,광주,-27
5,대구,-180
6,대전,-263
7,부산,-52
8,서울,-1745
9,세종시,93


In [71]:
total_doctors_diff.to_csv('/Users/haeunpark/mid_project_data/2021년대비2024년_지역별_의사수_증감량.csv', index=False)

In [14]:
# 시도별 의사수 증감량 막대 그래프 생성
fig = px.bar(total_doctors_diff.reset_index(),
             x='시도코드명', 
             y='총의사수', 
             title='2021년 대비 2024년 시도별 의사수 증감량',
             labels={'총의사수': '의사수 증감량'})

# 그래프 표시
fig.show()

In [77]:
# 시도별 병원 유형별 의사수 집계 (2024년도 데이터)
doctors_2024_by_region_type = df_2024_unique.groupby(['시도코드명', '의료수준'])['총의사수'].sum().unstack(fill_value=0)

# 시도별 병원 유형별 의사수 집계 (2021년도 데이터)
doctors_2021_by_region_type = df_2021_unique.groupby(['시도코드명', '의료수준'])['총의사수'].sum().unstack(fill_value=0)

# 시도별 병원 유형별 의사수 변화량 계산 (2024년도 - 2021년도)
doctors_diff_by_region_type = doctors_2024_by_region_type - doctors_2021_by_region_type

# 결측값(NaN)을 0으로 대체 (변화가 없거나 새로운 유형일 경우)
doctors_diff_by_region_type = doctors_diff_by_region_type.fillna(0)

# 결과 출력
print("2021년 대비 2024년 시도별 병원 유형별 의사수 변화량:")
doctors_diff_by_region_type = doctors_diff_by_region_type.reset_index()
doctors_diff_by_region_type

2021년 대비 2024년 시도별 병원 유형별 의사수 변화량:


의료수준,시도코드명,1차 의료,2차 의료,3차 의료,특수의료
0,강원,169,-145,-193,0
1,경기,2814,-314,-423,5
2,경남,202,40,-368,1
3,경북,155,18,0,1
4,광주,229,84,-341,0
5,대구,459,77,-735,0
6,대전,154,-406,-11,1
7,부산,539,-243,-353,0
8,서울,2721,-861,-3607,3
9,세종시,78,27,0,0


In [78]:
doctors_diff_by_region_type.to_csv('/Users/haeunpark/mid_project_data/2021년대비2024년_지역별_1차_2차_3차의사수_증감량.csv', index=False)

In [17]:
# 병원 유형별 의사수 변화량 그래프 생성
fig = px.bar(doctors_diff_by_region_type.reset_index(),
             x='시도코드명',  # 시도코드명
             y=['1차 의료', '2차 의료', '3차 의료'],  # 의료 수준별 의사수 변화량
             title='2021년 대비 2024년 시도별 병원 유형별 의사수 변화량',
             barmode='group')  # 그룹화된 막대 그래프


# 막대 사이의 간격 조정
fig.update_layout(bargap= 0.4,  # 시도 간의 막대 간격
                  bargroupgap=0.1)  # 동일한 시도의 그룹 내 막대 간격

# 그래프 표시
fig.show()

In [80]:
def categorize_region(region):
    if region in ['서울', '경기', '인천']:
        return '수도권'
    else:
        return '비수도권'

# 2024년 데이터에 지역 분류 추가
df_2024_unique['지역'] = df_2024_unique['시도코드명'].apply(categorize_region)

# 2021년 데이터에 지역 분류 추가
df_2021_unique['지역'] = df_2021_unique['시도코드명'].apply(categorize_region)

# 수도권 및 비수도권 의사수 집계 (2024년도)
total_doctors_2024_by_region = df_2024_unique.groupby('지역')['총의사수'].sum()

# 수도권 및 비수도권 의사수 집계 (2021년도)
total_doctors_2021_by_region = df_2021_unique.groupby('지역')['총의사수'].sum()

# 수도권 및 비수도권 의사수 증감량 계산 (2024년도 - 2021년도)
total_doctors_diff_by_region = total_doctors_2024_by_region - total_doctors_2021_by_region

# 결측값(NaN)을 0으로 대체 (변화가 없는 경우)
total_doctors_diff_by_region = total_doctors_diff_by_region.fillna(0)

# 증감률 계산: (2024년도 - 2021년도) / 2021년도 * 100
total_doctors_percentage_change = ((total_doctors_2024_by_region - total_doctors_2021_by_region) / total_doctors_2021_by_region) * 100

# 결과 출력
print("2021년 대비 2024년 수도권 및 비수도권 의사수 증감량:")
total_doctors_diff_by_region=total_doctors_diff_by_region.reset_index()
total_doctors_diff_by_region

2021년 대비 2024년 수도권 및 비수도권 의사수 증감량:


/var/folders/zj/p3cl2py51ldfrgsbtfp578x40000gn/T/ipykernel_41520/1674813092.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/zj/p3cl2py51ldfrgsbtfp578x40000gn/T/ipykernel_41520/1674813092.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,지역,총의사수
0,비수도권,-966
1,수도권,852


In [81]:
total_doctors_diff_by_region.to_csv('/Users/haeunpark/mid_project_data/2021년대비2024년_수도권_비수도권_의사수_증감량.csv', index=False)